<a href="https://colab.research.google.com/github/ollihansen90/Mathe-SH/blob/main/Wordketten_Termin1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wortketten


## Fragen?
Solltet ihr Fragen zum Code oder Probleme mit Colab haben, schickt uns gerne eine Mail:

*   h.hansen@uni-luebeck.de
*   mika.kohlhammer@student.uni-luebeck.de
*   gun.ingwersen@student.uni-luebeck.de

## 🧩 Wortketten: von Wort zu Wort mit nur einem Buchstaben

In dieser Aufgabe beschäftigen wir uns mit sogenannten **Wortketten** (auch *Wortleitern* genannt).

Eine Wortkette ist eine Abfolge von Wörtern, bei der sich **von einem Wort zum nächsten immer genau ein Buchstabe ändert**.  
Alle Wörter müssen dabei gültige Wörter sein.

### Beispiel
Hund → Wund → Wand → Wald → bald → Ball → Fall → Fell → Feld

Zwischen jeweils zwei aufeinanderfolgenden Wörtern unterscheidet sich **nur ein Buchstabe**.

### Ziel dieses Notebooks

In diesem Notebook werden wir Schritt für Schritt lernen,

- wie man Wörter in Python speichert und vergleicht,
- wie man überprüft, ob sich zwei Wörter nur in einem Buchstaben unterscheiden,
- und wie man mit Python selbst solche Wortketten findet oder erstellt.

Die folgenden Unterlagen dienen als Vorkenntnisse zur Bearbeitung dieses Notebooks:
- Termin 1-7

👉 Am Ende können wir ein eigenes kleines Wortspiel programmieren!



### Distanzfunktion
## 🔤 Ähnlichkeit zwischen zwei Wörtern berechnen

Um herauszufinden, **wie ähnlich sich zwei Wörter sind**, verwenden wir eine Funktion namens `distanz`.  
Sie zählt, **an wie vielen Stellen sich zwei Wörter unterscheiden**.

**Wichtige Voraussetzung:**  
Beide Wörter müssen **gleich lang** sein.

---

## 🧠 Idee der Funktion

- Die beiden Wörter werden zuerst in **Großbuchstaben** umgewandelt  
  (damit z. B. `m` und `M` als gleich gelten).
- Danach werden die Wörter **Buchstabe für Buchstabe** verglichen.
- Jedes Mal, wenn sich zwei Buchstaben unterscheiden, wird ein Zähler um **1 erhöht**.
- Am Ende gibt die Funktion die Anzahl der Unterschiede zurück.

In [ ]:
def distanz(w1, w2):
    w1 = w1.upper()
    w2 = w2.upper()
    ausgabe = 0
    # Hier eigenen Code einbauen

    return ausgabe

print(distanz("Mund", "Mond")) # Ausgabe: 1
print(distanz("Fell", "Ball")) # Ausgabe: 2
print(distanz("Ball", "Mond")) # Ausgabe: 4

## Gruppieren von ähnlichen Wörtern 📝✨

Zwei Wörter gelten als **ähnlich** 🤝, wenn sie sich **in genau einem Buchstaben unterscheiden** 🔤.  
Die Funktion `distanz(wort1, wort2)` prüft diese Ähnlichkeit ✅.

Die Wörter werden in **Gruppen** 🗂️ zusammengefasst:

- Jedes Wort wird mit allen Wörtern der bestehenden Gruppen verglichen 🔎.
- Passt es zu mindestens einem Wort einer Gruppe (Distanz = 1) ➡️, wird es dieser Gruppe hinzugefügt.
- Passt es zu keiner Gruppe ❌, wird eine neue Gruppe erstellt .

### Beispielwortliste 📋

```python
wortliste = ["Wand", "Hund", "Hand", "Feld", "Fell", "Fall", "Ball", "bald"]

In [ ]:
from tqdm.auto import tqdm
wortliste = list(map(lambda x: x.upper(), ["Wand", "Hund", "Hand",  "Feld", "Fell", "Fall", "Ball", "bald"]))

gruppenliste = [[wortliste[0]]]
for wort in tqdm(wortliste[1:]):
    enthalten = False

    if not enthalten:
        gruppenliste.append([wort])
print(gruppenliste)

## ⚠️ Problem der greedy Gruppierung von Wortlisten

Die verwendete Strategie ist **greedy** und **reihenfolgeabhängig**:

- Wörter werden nacheinander verarbeitet.
- Ein neues Wort wird **der ersten Gruppe zugeordnet**, zu der es Distanz 1 hat.
- Falls keine passende Gruppe existiert, wird eine **neue Gruppe** erstellt.

### 📌 Zentrales Problem

Diese Vorgehensweise garantiert **keine maximalen oder korrekten Wortlisten**, da:

- Ein Wort zu **mehreren Gruppen passen könnte**, aber nur einer zugeordnet wird.
- **Frühere Entscheidungen nicht revidiert** werden.
- Zusammenhängende Wörter dadurch **auf mehrere Gruppen aufgespalten** werden.

### 🔍 Beispiel

Aus einer zusammenhängenden Wortkette wie:

`HUND – HAND – WAND`

entstehen getrennte Gruppen:

- `['WAND', 'HAND']`
- `['HUND', 'HAND']`

obwohl alle Wörter über Distanz 1 verbunden sind.

### 🏁 Fazit

Das Problem entspricht der Suche nach **zusammenhängenden Komponenten in einem Graphen**  
(Knoten = Wörter, Kante = Distanz 1).

Eine greedy Gruppierung ist hierfür **nicht geeignet**.

## 🧠 Idee: Gruppen nachträglich zusammenführen

Beim ersten Durchlauf ordnen wir jedes Wort **vorläufig** einer oder mehreren Gruppen zu.
Dabei kann es passieren, dass **zusammengehörige Wörter in verschiedenen Gruppen landen**.

### 📐 Grundidee

- Ein neues Wort kann zu **mehr als einer Gruppe passen**  
  (weil es sich nur in einem Buchstaben unterscheidet).
- In diesem Fall gehören diese Gruppen **eigentlich zusammen**.
- Deshalb **verschmelzen wir Gruppen**, die ein gemeinsames Wort haben.

### 🔍 Vorgehen Schritt für Schritt

1. Wir gehen die Wörter **nacheinander** durch.
2. Für jedes Wort prüfen wir:
   - Zu welchen bestehenden Gruppen passt es  
     (Distanz genau 1 zu mindestens einem Wort der Gruppe)?
3. Es gibt drei Fälle:

#### Fall 1: Passt zu keiner Gruppe
- Wir erstellen eine **neue Gruppe**, die nur dieses Wort enthält.

#### Fall 2: Passt zu genau einer Gruppe
- Das Wort wird **an diese Gruppe angehängt**.

#### Fall 3: Passt zu mehreren Gruppen
- Diese Gruppen gehören zusammen.
- Wir:
  - sammeln alle Wörter dieser Gruppen,
  - entfernen doppelte Einträge,
  - erstellen daraus **eine neue gemeinsame Gruppe**,
  - löschen die alten Gruppen.

### 🏁 Ergebnis

- Am Ende enthält jede Gruppe Wörter,
  die **über Buchstabenänderungen miteinander verbunden** sind.
- Die Reihenfolge der Wörter spielt jetzt **keine Rolle mehr**.
- Jede Gruppe entspricht einer zusammenhängenden „Wortfamilie“.

### 🧠 Merksatz

> Wenn ein Wort mehrere Gruppen verbindet,  
> dann müssen diese Gruppen zusammengeführt werden.


In [ ]:
liste = [["WAND", "HAND"], ["HUND", "HAND"], ["FELD", "FELL", "FALL", "BALL", "BALD"]]

